In [43]:
# imports
from octis.models.LDA import LDA
from octis.dataset.dataset import Dataset
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.dataset.dataset import Dataset
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.optimization.optimizer import Optimizer
optimizer=Optimizer()
import time
import en_core_web_sm
nlp = en_core_web_sm.load()
dataset = Dataset()
dataset.load_custom_dataset_from_folder("NL_OPT")
docs = [" ".join(words) for words in dataset.get_corpus()]

In [44]:
from octis.models.NeuralLDA import NeuralLDA
import inspect
print(inspect.getsource(NeuralLDA))

class NeuralLDA(AVITM):
    def __init__(
        self, num_topics=10, activation='softplus', dropout=0.2,
        learn_priors=True, batch_size=64, lr=2e-3, momentum=0.99,
        solver='adam', num_epochs=100, reduce_on_plateau=False, prior_mean=0.0,
        prior_variance=None, num_layers=2, num_neurons=100, num_samples=10,
            use_partitions=True):
        super().__init__(
            num_topics=num_topics, model_type='LDA', activation=activation,
            dropout=dropout, learn_priors=learn_priors, batch_size=batch_size,
            lr=lr, momentum=momentum, solver=solver, num_epochs=num_epochs,
            reduce_on_plateau=reduce_on_plateau, prior_mean=prior_mean,
            prior_variance=prior_variance, num_layers=num_layers,
            num_neurons=num_neurons, num_samples=num_samples,
            use_partitions=use_partitions)

    def train_model(self, dataset, hyperparameters=None, top_words=10):
        return super().train_model(
            dataset=dataset,

In [45]:
# num_topics=100, distributed=False, chunksize=2000,
#         passes=1, update_every=1, alpha="symmetric", eta=None, decay=0.5,
#         offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001,
#             random_state=None)

In [46]:
# num_topics=10, activation='softplus', dropout=0.2,
#         learn_priors=True, batch_size=64, lr=2e-3, momentum=0.99,
#         solver='adam', num_epochs=100, reduce_on_plateau=False, prior_mean=0.0,
#         prior_variance=None, num_layers=2, num_neurons=100, num_samples=10,
#             use_partitions=True

In [47]:
#SETTING METRICS
coherence = Coherence(texts=dataset.get_corpus(), measure = 'c_npmi')
diversity = TopicDiversity(topk=10)

#RUN CONFIGURATION
optimization_runs=12
model_runs=3

# LDA

In [23]:
model = LDA()

search_space = {
    "num_topics": [10,20,30,40,50],
    "alpha": ['symmetric','asymmetric']
}

In [24]:
#OPTIMIZATION
start = time.time()
optimization_result = optimizer.optimize(
    model, dataset, coherence, search_space, number_of_call=optimization_runs, 
    model_runs=model_runs, save_models=True, 
    extra_metrics=[diversity], # to keep track of other metrics
    save_path='results2/test_NLDA2//')
end = time.time()
duration = end - start
optimization_result.save_to_csv('results_LDA3.csv')
print('Optimizing model took: ' + str(round(duration)) + ' seconds.')

Current call:  0


AttributeError: 'NoneType' object has no attribute 'get_partitioned_corpus'

In [48]:
model = NeuralLDA(num_topics=10)
search_space = {
    "activation": ['softplus','sigmoid', 'relu'],
    "num_layers" : [1,2,4,8], 
    "num_neurons": [50,100,200,300]}

In [49]:
#OPTIMIZATION
start = time.time()
optimization_result = optimizer.optimize(
    model, dataset, coherence, search_space, number_of_call=optimization_runs, 
    model_runs=model_runs, save_models=True, 
    extra_metrics=[diversity], # to keep track of other metrics
    save_path='results/test_neuralLDA//')
end = time.time()
duration = end - start
optimization_result.save_to_csv("results_neuralLDA.csv")
print('Optimizing model took: ' + str(round(duration)) + ' seconds.')

Current call:  0


/home/pspaargaren/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1320: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


Epoch: [1/100]	Samples: [19058/1905800]	Train Loss: 1115.908307042974	Time: 0:00:05.344867


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
optimizer = Optimizer()
json_path = 'results2/test_NLDA2/result.json'
optimizer.resume_optimization(json_path)

In [ ]:
import json
results = json.load(open("results2/test_LDA2/result.json",'r'))
results.keys()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.xlabel('Iteration')
plt.ylabel('Coherence score (c_npmi)')
plt.title('Median coherence score per iteration')
plt.plot(results["f_val"])
plt.show()

In [ ]:
results[ 'f_val'].index(max(results[ 'f_val']))

In [ ]:
print([results['x_iters'][parameter][0] for parameter in results['x_iters'].keys()])